# DLT Pipeline
### Streaming table

In [0]:
import dlt
from pyspark.sql.functions import *

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
DLTImportException                        Traceback (most recent call last)
File <command-8538556697469920>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import *

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
     70     self._should_hint = True
---> 71 module = self._original_builtins_import(name, *args, **kwargs)
     72 if (fname := fname or get_allowed_file_name_or_none(module)) is not None:
     73     mtime = mtime or os.stat(fname).st_mtime

In [0]:
#Expectation
my_rules = {
    "rule1" : "product_id is NOT NULL",
    "rule2" : "product_name is NOT NULL"
}

In [0]:
@dlt.table
@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
    df = spark.readStream.format("delta") \
        .option("skipChangeCommits", "true") \
        .table("databricks_cata.silver.products_silvers")
    return df

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

streaming view

In [0]:
@dlt.view

def DimProducts_view():
    df = spark.readStream.table("Live.DimProducts_stage")
    return df


Dim products

In [0]:
dlt.create_streaming_table("DimProducts")

In [0]:
dlt.apply_changes(
    target = "DimProducts",
    source = "Live.DimProducts_view",
    keys = ["product_id"],
    sequence_by = col("product_id"),
    stored_as_scd_type =2
)